# Python HW monitoring

Based on Tutorial [Hardware and System Information in Python](https://www.thepythoncode.com/article/get-hardware-system-information-python)

In [21]:
import psutil
import platform
from datetime import datetime
import GPUtil
from tabulate import tabulate
from hw import get_readable_size

In [22]:
print("="*40, "System Information", "="*40)
uname = platform.uname()
print(f"System: {uname.system}")
print(f"Node Name: {uname.node}")
print(f"Release: {uname.release}")
print(f"Version: {uname.version}")
print(f"Machine: {uname.machine}")
print(f"Processor: {uname.processor}")

======================================== System Information ========================================
System: Windows
Node Name: Beast
Release: 10
Version: 10.0.22621
Machine: AMD64
Processor: AMD64 Family 23 Model 113 Stepping 0, AuthenticAMD


In [23]:
# Boot Time
print("="*40, "Boot Time", "="*40)
boot_time_timestamp = psutil.boot_time()
bt = datetime.fromtimestamp(boot_time_timestamp)
print(f"Boot Time: {bt.year}-{bt.month}-{bt.day} {bt.hour}:{bt.minute}:{bt.second}")

======================================== Boot Time ========================================
Boot Time: 2023-3-6 7:35:54


In [24]:
# let's print CPU information
print("="*40, "CPU Info", "="*40)
# number of cores
print("Physical cores:", psutil.cpu_count(logical=False))
print("Total cores:", psutil.cpu_count(logical=True))
# CPU frequencies
cpufreq = psutil.cpu_freq()
print(f"Max Frequency: {cpufreq.max:.2f}Mhz")
print(f"Min Frequency: {cpufreq.min:.2f}Mhz")
print(f"Current Frequency: {cpufreq.current:.2f}Mhz")
# CPU usage
print("CPU Usage Per Core:")
for i, percentage in enumerate(psutil.cpu_percent(percpu=True, interval=1)):
    print(f"Core {i}: {percentage}%")
print(f"Total CPU Usage: {psutil.cpu_percent()}%")

======================================== CPU Info ========================================
Physical cores: 12
Total cores: 24
Max Frequency: 4276.00Mhz
Min Frequency: 0.00Mhz
Current Frequency: 4276.00Mhz
CPU Usage Per Core:
Core 0: 14.1%
Core 1: 23.4%
Core 2: 34.3%
Core 3: 12.5%
Core 4: 3.1%
Core 5: 4.7%
Core 6: 7.9%
Core 7: 3.1%
Core 8: 3.1%
Core 9: 14.1%
Core 10: 4.6%
Core 11: 0.0%
Core 12: 0.0%
Core 13: 0.0%
Core 14: 0.0%
Core 15: 1.6%
Core 16: 0.0%
Core 17: 0.0%
Core 18: 0.0%
Core 19: 3.1%
Core 20: 0.0%
Core 21: 4.7%
Core 22: 1.6%
Core 23: 1.6%
Total CPU Usage: 5.2%


In [25]:
# Memory Information
print("="*40, "Memory Information", "="*40)
# get the memory details
svmem = psutil.virtual_memory()
print(f"Total: {get_readable_size(svmem.total)}")
print(f"Available: {get_readable_size(svmem.available)}")
print(f"Used: {get_readable_size(svmem.used)}")
print(f"Percentage: {svmem.percent}%")
print("="*20, "SWAP", "="*20)
# get the swap memory details (if exists)
swap = psutil.swap_memory()
print(f"Total: {get_readable_size(swap.total)}")
print(f"Free: {get_readable_size(swap.free)}")
print(f"Used: {get_readable_size(swap.used)}")
print(f"Percentage: {swap.percent}%")

======================================== Memory Information ========================================
Total: 63.92GB
Available: 30.33GB
Used: 33.59GB
Percentage: 52.5%
==================== SWAP ====================
Total: 5.93GB
Free: -18648764416.00B
Used: 23.29GB
Percentage: 393.1%


In [26]:
# Disk Information
print("="*40, "Disk Information", "="*40)
print("Partitions and Usage:")
# get all disk partitions
partitions = psutil.disk_partitions()

# Docker containers shows up!!!!!
for partition in partitions:
    print(f"=== Device: {partition.device} ===")
    print(f"  Mountpoint: {partition.mountpoint}")
    print(f"  File system type: {partition.fstype}")
    try:
        partition_usage = psutil.disk_usage(partition.mountpoint)
    except PermissionError:
        # this can be catched due to the disk that
        # isn't ready
        continue
    print(f"  Total Size: {get_readable_size(partition_usage.total)}")
    print(f"  Used: {get_readable_size(partition_usage.used)}")
    print(f"  Free: {get_readable_size(partition_usage.free)}")
    print(f"  Percentage: {partition_usage.percent}%")

    
# get IO statistics since boot
disk_io = psutil.disk_io_counters()
print(f"Total read: {get_readable_size(disk_io.read_bytes)}")
print(f"Total write: {get_readable_size(disk_io.write_bytes)}")

======================================== Disk Information ========================================
Partitions and Usage:
=== Device: C:\ ===
  Mountpoint: C:\
  File system type: NTFS
  Total Size: 930.73GB
  Used: 763.48GB
  Free: 167.25GB
  Percentage: 82.0%
=== Device: D:\ ===
  Mountpoint: D:\
  File system type: NTFS
  Total Size: 3.64TB
  Used: 1.17TB
  Free: 2.47TB
  Percentage: 32.2%
=== Device: E:\ ===
  Mountpoint: E:\
  File system type: NTFS
  Total Size: 446.58GB
  Used: 110.11MB
  Free: 446.47GB
  Percentage: 0.0%
=== Device: F:\ ===
  Mountpoint: F:\
  File system type: NTFS
  Total Size: 17.44GB
  Used: 52.92MB
  Free: 17.39GB
  Percentage: 0.3%
=== Device: Q:\ ===
  Mountpoint: Q:\
  File system type: NTFS
  Total Size: 931.50GB
  Used: 246.10GB
  Free: 685.40GB
  Percentage: 26.4%
=== Device: R:\ ===
  Mountpoint: R:\
  File system type: NTFS
  Total Size: 3.64TB
  Used: 15.48GB
  Free: 3.62TB
  Percentage: 0.4%
Total read: 89.28GB
Total write: 124.31GB


In [27]:
# Network information
print("="*40, "Network Information", "="*40)
# get all network interfaces (virtual and physical)
if_addrs = psutil.net_if_addrs()
for interface_name, interface_addresses in if_addrs.items():
    for address in interface_addresses:
        print(f"=== Interface: {interface_name} ===")
        if str(address.family) == 'AddressFamily.AF_INET':
            print(f"  IP Address: {address.address}")
            print(f"  Netmask: {address.netmask}")
            print(f"  Broadcast IP: {address.broadcast}")
        elif str(address.family) == 'AddressFamily.AF_PACKET':
            print(f"  MAC Address: {address.address}")
            print(f"  Netmask: {address.netmask}")
            print(f"  Broadcast MAC: {address.broadcast}")
# get IO statistics since boot
net_io = psutil.net_io_counters()
print(f"Total Bytes Sent: {get_readable_size(net_io.bytes_sent)}")
print(f"Total Bytes Received: {get_readable_size(net_io.bytes_recv)}")

======================================== Network Information ========================================
=== Interface: Local Area Connection ===
=== Interface: Local Area Connection ===
=== Interface: Local Area Connection ===
=== Interface: Ethernet ===
=== Interface: Ethernet ===
=== Interface: Ethernet ===
=== Interface: Local Area Connection* 1 ===
=== Interface: Local Area Connection* 1 ===
=== Interface: Local Area Connection* 1 ===
=== Interface: Local Area Connection* 10 ===
=== Interface: Local Area Connection* 10 ===
=== Interface: Local Area Connection* 10 ===
=== Interface: WiFi ===
=== Interface: WiFi ===
=== Interface: WiFi ===
=== Interface: Bluetooth Network Connection 2 ===
=== Interface: Bluetooth Network Connection 2 ===
=== Interface: Bluetooth Network Connection 2 ===
=== Interface: vEthernet (WSL) ===
=== Interface: vEthernet (WSL) ===
=== Interface: vEthernet (WSL) ===
=== Interface: Loopback Pseudo-Interface 1 ===
=== Interface: Loopback Pseudo-Interface 1 ===
Tot

## GPU info 

GPUtil is a Python module for getting the GPU status for NVIDIA GPUs only.

In [28]:
# GPU information
import GPUtil
from tabulate import tabulate
print("="*40, "GPU Details", "="*40)
gpus = GPUtil.getGPUs()
list_gpus = []
for gpu in gpus:
    # get the GPU id
    gpu_id = gpu.id
    # name of GPU
    gpu_name = gpu.name
    # get % percentage of GPU usage of that GPU
    gpu_load = f"{gpu.load*100}%"
    # get free memory in MB format
    gpu_free_memory = f"{gpu.memoryFree}MB"
    # get used memory
    gpu_used_memory = f"{gpu.memoryUsed}MB"
    # get total memory
    gpu_total_memory = f"{gpu.memoryTotal}MB"
    # get GPU temperature in Celsius
    gpu_temperature = f"{gpu.temperature} °C"
    gpu_uuid = gpu.uuid
    list_gpus.append((
        gpu_id, gpu_name, gpu_load, gpu_free_memory, gpu_used_memory,
        gpu_total_memory, gpu_temperature, gpu_uuid
    ))

print(tabulate(list_gpus, headers=("id", "name", "load", "free memory", "used memory", "total memory",
                                   "temperature", "uuid")))

======================================== GPU Details ========================================
  id  name                           load    free memory    used memory    total memory    temperature    uuid
----  -----------------------------  ------  -------------  -------------  --------------  -------------  ----------------------------------------
   0  NVIDIA GeForce RTX 2060 SUPER  1.0%    4129.0MB       3889.0MB       8192.0MB        45.0 °C        GPU-2f3a2dd6-4339-dd16-bfe2-4b897b197f1d


In [29]:
for i in range(len(psutil.sensors_temperatures(fahrenheit=False)[ 'coretemp'])):
    print(str(psutil.sensors_temperatures(fahrenheit=False)[ 'coretemp'][i].label) + " has a temp of " + str(psutil.sensors_temperatures(fahrenheit=False)[ 'coretemp'][i].current) + "°C")
 

AttributeError: module 'psutil' has no attribute 'sensors_temperatures'

In [30]:
print("All temps",psutil.sensors_temperatures())
print("Fans",psutil.sensors_fans())
print("Battery",psutil.sensors_battery())
psutil.disk_usage("/home/")



AttributeError: module 'psutil' has no attribute 'sensors_temperatures'

In [31]:
psutil.getloadavg()

(0.0, 0.0, 0.0)

## BEST OF

In [20]:
# Boot Time
print("="*40, "Boot Time", "="*40)
boot_time_timestamp = psutil.boot_time()
bt = datetime.fromtimestamp(boot_time_timestamp)
print(f"Boot Time: {bt.year}-{'{:02d}'.format(bt.month)}-{bt.day} {bt.hour}:{bt.minute}:{bt.second}")

# CPU 
print("="*40, "CPU Info", "="*40)
print(f"CPU Usage: {psutil.cpu_percent()}%")
cpu_core_temps = []
for i in range(len(psutil.sensors_temperatures()[ 'coretemp'])):
    cpu_core_temps.append(psutil.sensors_temperatures()[ 'coretemp'][i].current)
print("CPU temp mean:", (sum(cpu_core_temps) / len(cpu_core_temps)), "°C")

# RAM 
print("="*40, "RAM / Memory Info", "="*40)
print(f"RAM use: {svmem.percent}%")

# GPU
print("="*40, "GPU Details", "="*40)
gpus = GPUtil.getGPUs()

if len(gpus) < 1:
    print("No GPU found")
else:
    for gpu in gpus:
        # get % percentage of GPU usage of that GPU
        gpu_load = f"{gpu.load*100}%"
        # get GPU temperature in Celsius
        gpu_temperature = f"{gpu.temperature} °C"
        print("ID:", gpu.id)
        print("Name:", gpu.name)
        print("Load:", gpu_load)
        print("Temp:", gpu_temperature)

# Disk Information
print("="*40, "Disk Information", "="*40)
psutil.disk_usage("/")
print("Partitions:")
# get all disk partitions
partitions = psutil.disk_partitions()

for partition in partitions[:1]:
    print(f"=== Device: {partition.device} ===")
    print(f"  Mountpoint: {partition.mountpoint}")
    try:
        partition_usage = psutil.disk_usage(partition.mountpoint)
    except PermissionError:
        # this can be catched due to the disk that
        # isn't ready
        continue
    print(f"  Total Size: {get_readable_size(partition_usage.total)}")
    print(f"  Used: {get_readable_size(partition_usage.used)}")
    print(f"  Free: {get_readable_size(partition_usage.free)}")




======================================== Boot Time ========================================
Boot Time: 2023-03-6 7:35:54
======================================== CPU Info ========================================
CPU Usage: 4.9%


AttributeError: module 'psutil' has no attribute 'sensors_temperatures'